In [1]:
from azureml.core import Workspace
from azureml.core.authentication import MsiAuthentication
msi_auth = MsiAuthentication()

ws = Workspace(subscription_id="7704663e-8944-4300-b951-6462d41ab638", # put in .json
resource_group="arg-rt-pacedev-SpaceTech", # also in .json
workspace_name="aml-WS-spacetech001", # also
auth=msi_auth
)

keyvault = ws.get_default_keyvault()
conn_str = keyvault.get_secret("asartpacedevstvdsm001-connstr")

# Baseline Dataset

In [2]:
import datetime
from dateutil.relativedelta import relativedelta
from vdsm_util import get_blob_client
import pickle as pkl
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
import traceback

veg_type = 'npv'

training_path = 'training/traininginput_{}_v2'.format(veg_type)

blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")

baseline_df = pd.DataFrame()

blob_names = container_client.list_blobs(name_starts_with=f'training/traininginput_{veg_type}_v2')

for blob_name in blob_names:
    blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
    df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
    baseline_df = baseline_df.append(df)

baseline_df.drop(columns=['actual_3m', 'actual_6m', 'actual_9m', 'actual_12m', 'lat_4326', 'lon_4326', 'year'], inplace=True)

output = baseline_df.to_csv(index=False, encoding='utf-8', header=True)

blob_client = get_blob_client(filename=f'data_drift/baseline/{veg_type}.csv', conn_str=conn_str)
blob_client.upload_blob(output, overwrite=True, blob_type='BlockBlob')

{'etag': '"0x8D9EACCF0BAA133"',
 'last_modified': datetime.datetime(2022, 2, 8, 6, 33, 38, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'\x190\xc4%p\x0b\xc5['),
 'client_request_id': '0c853f46-88a9-11ec-8b2c-1bbaf30259bc',
 'request_id': 'd5c5d1b9-101e-00bb-41b5-1ce193000000',
 'version': '2020-06-12',
 'version_id': '2022-02-08T06:33:38.2636611Z',
 'date': datetime.datetime(2022, 2, 8, 6, 33, 37, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

# Pipeline for monthly target dataset

In [3]:
import datetime
from dateutil.relativedelta import relativedelta
from vdsm_util import get_blob_client
import pickle as pkl
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
import traceback

end_month = '2021-12'
veg_type = 'npv'

blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")

end_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1)
start_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1) + relativedelta(months=-12)

months = []
month_dt = start_month_dt
while month_dt < end_month_dt:
    month_dt = month_dt + relativedelta(months=1)
    months.append(str(month_dt)[:7])

target_df = pd.DataFrame()

for month in months:

    blob_names = container_client.list_blobs(name_starts_with=f'{month[:4]}/{month[5:]}/fused/{veg_type}_v1')

    for blob_name in blob_names:
        blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
        df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
        if datetime.date(int(end_month[:4]), int(end_month[5:]), 1) >= datetime.date(2021, 12, 1):
            try:
                df = df.sample(n=100)
            except:
                pass
        target_df = target_df.append(df)

target_df.drop(columns=['filename', 'lat', 'lon', 'year'], inplace=True)
target_df.insert(0, 'month', target_df.iloc[:, 0:12].values.argmax(axis=1) + 1)
target_df.drop(columns=['month1','month2','month3','month4','month5','month6','month7','month8','month9','month10','month11','month12'], inplace=True)

output = target_df.to_csv(index=False, encoding='utf-8', header=True)

blob_client = get_blob_client(filename=f'data_drift/target/{end_month}/{veg_type}.csv', conn_str=conn_str)
blob_client.upload_blob(output, overwrite=True, blob_type='BlockBlob')

Traceback (most recent call last):
  File "<ipython-input-3-e33ad294e873>", line 36, in <module>
    df = df.sample(n=100)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/generic.py", line 4970, in sample
    locs = rs.choice(axis_length, size=n, replace=replace, p=weights)
  File "mtrand.pyx", line 946, in numpy.random.mtrand.RandomState.choice
ValueError: Cannot take a larger sample than population when 'replace=False'

Traceback (most recent call last):
  File "<ipython-input-3-e33ad294e873>", line 36, in <module>
    df = df.sample(n=100)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/generic.py", line 4970, in sample
    locs = rs.choice(axis_length, size=n, replace=replace, p=weights)
  File "mtrand.pyx", line 946, in numpy.random.mtrand.RandomState.choice
ValueError: Cannot take a larger sample than population when 'replace=False'

Traceback (most recent call last):
  File "<ipython-input-3-e33ad294e873>", line 36, in <mod

In [ ]:
output = target_df.to_csv(index=False, encoding='utf-8', header=True)

blob_client = get_blob_client(filename=f'data_drift/target/{end_month}/{veg_type}.csv', conn_str=conn_str)
blob_client.upload_blob(output, overwrite=True, blob_type='BlockBlob')